In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression

In [3]:
bank_data = pd.read_csv('dataset/Bank_Personal_Loan_Modelling.csv')

bank_data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [4]:
X = bank_data.drop('CreditCard', axis=1)

Y = bank_data['CreditCard']

x_train, x_hold_out, x_test = np.split(X, [int(.7*len(X)), int(.9*len(X))]) #70% 20% 10%

y_train, y_hold_out, y_test = np.split(Y, [int(.7*len(Y)), int(.9*len(Y))])

In [5]:
clf1 = KNeighborsClassifier(n_neighbors=10)
clf2 = RandomForestClassifier(n_estimators=50)
clf3 = GaussianNB()

for clf in (clf1, clf2, clf3):
    clf.fit(x_train, y_train)

In [6]:
def get_prediction(x, y):
    pred_result = pd.DataFrame()
    
    i = 1
    for clf in (clf1, clf2, clf3):
        
        y_pred = clf.predict(x)
        
        print(clf.__class__.__name__, accuracy_score(y, y_pred))
        
        pred_result.insert(i - 1, 'y_pred_' + str(i), y_pred)
        
        i += 1
        
    return pred_result

In [7]:
pred_result = get_prediction(x_hold_out, y_hold_out)

KNeighborsClassifier 0.679
RandomForestClassifier 0.72
GaussianNB 0.749


In [8]:
pred_result

,y_pred_1,y_pred_2,y_pred_3
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
995,0,0,0
996,0,0,0
997,0,0,0
998,0,0,0


In [9]:
x_stack_train = pred_result

y_stack_train = y_hold_out

clf_stack = LogisticRegression(solver='lbfgs', C=1, max_iter=200)

clf_stack.fit(x_stack_train, y_stack_train)

pred_result_test = get_prediction(x_test, y_test)

KNeighborsClassifier 0.67
RandomForestClassifier 0.708
GaussianNB 0.734


In [10]:
x_stack_test = pred_result_test

y_stack_pred = clf_stack.predict(x_stack_test)

In [11]:
accuracy_score(y_test, y_stack_pred)

0.734